In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from ripser import Rips
import persim
from keras.models import load_model

model = load_model("lstm.keras", compile=False)

In [2]:
tickers_df = pd.read_csv("../feature_extraction/valid_tickers.csv")
tickers = tickers_df["ticker"].tolist()

In [25]:
end_date = datetime.now().date()
start_date = end_date - timedelta(days=61)

tickers_df = pd.read_csv("../feature_extraction/valid_tickers.csv")
tickers = tickers_df["ticker"].tolist()

sp500_data = yf.download(tickers, start=start_date, end=end_date, group_by="ticker")

/var/folders/c5/nwrhnbhs5ks1ssf190f4b49r0000gn/T/ipykernel_54047/3948707593.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  sp500_data = yf.download(tickers, start=start_date, end=end_date, group_by="ticker")
[*********************100%***********************]  474 of 474 completed


In [26]:
df_finance = sp500_data.copy()
df_finance.columns = [f"{col[0]}_{col[1]}" for col in df_finance.columns]

df_finance = df_finance.reset_index()
df_finance = df_finance.set_index('Date')

In [27]:
window_rsi = 14
window_vol = 21

def compute_rsi(series, window):
    delta = series.diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    roll_up = pd.Series(gain, index=series.index).rolling(window=window).mean()
    roll_down = pd.Series(loss, index=series.index).rolling(window=window).mean()
    
    RS = np.where(roll_down == 0, np.inf, roll_up / roll_down)
    RSI = 100.0 - (100.0 / (1.0 + RS))
    
    return RSI

for ticker in tickers:
    close_col = f"{ticker}_Close"
    volume_col = f"{ticker}_Volume"

    if close_col not in df_finance.columns or volume_col not in df_finance.columns:
        continue

    df_finance[f"{ticker}_RSI_14"] = compute_rsi(df_finance[close_col], window=window_rsi)
    
    log_return = np.log(1 + df_finance[close_col].pct_change())
    df_finance[f"{ticker}_Log_Return"] = log_return
    df_finance[f"{ticker}_Volatility_21"] = log_return.rolling(window=window_vol).std()
    
    df_finance[f"{ticker}_Volume_Z"] = (df_finance[volume_col] - df_finance[volume_col].rolling(window_vol).mean()) / df_finance[volume_col].rolling(window_vol).std()
# Drop first row due to NaN values
df_finance = df_finance.iloc[1:]

/var/folders/c5/nwrhnbhs5ks1ssf190f4b49r0000gn/T/ipykernel_54047/2417404046.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_finance[f"{ticker}_RSI_14"] = compute_rsi(df_finance[close_col], window=window_rsi)
/var/folders/c5/nwrhnbhs5ks1ssf190f4b49r0000gn/T/ipykernel_54047/2417404046.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_finance[f"{ticker}_Log_Return"] = log_return
/var/folders/c5/nwrhnbhs5ks1ssf190f4b49r0000gn/T/ipykernel_54047/2417404046.py:27: PerformanceWarning: DataFrame is highly fragmented.  This 

/var/folders/c5/nwrhnbhs5ks1ssf190f4b49r0000gn/T/ipykernel_54047/2417404046.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_finance[f"{ticker}_Volatility_21"] = log_return.rolling(window=window_vol).std()
/var/folders/c5/nwrhnbhs5ks1ssf190f4b49r0000gn/T/ipykernel_54047/2417404046.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_finance[f"{ticker}_Volume_Z"] = (df_finance[volume_col] - df_finance[volume_col].rolling(window_vol).mean()) / df_finance[volume_col].rolling(window_vol).std()
/var/folders/c5/nwrhnbhs5ks1s

In [31]:
window_wasserstein = 30
eps = 0.5  
maxdim = 2 
rips = Rips(maxdim=maxdim, verbose=False)

def betti_numbers_at_scale(diagrams, eps):
    bettis = []
    for dgm in diagrams:
        alive = np.sum((dgm[:, 0] <= eps) & (dgm[:, 1] > eps))
        bettis.append(int(alive))
    return bettis

first_ticker = list(tickers)[0]

log_returns = df_finance[f"{first_ticker}_Log_Return"].dropna().values

persistences = []
betti_list = []
wasserstein_list = []

for start in range(0, len(log_returns) - window_wasserstein + 1):
    window_data = log_returns[start:start + window_wasserstein]
    diagrams = rips.fit_transform(window_data)
    persistences.append(diagrams)

    bettis = betti_numbers_at_scale(diagrams, eps=eps)
    while len(bettis) < 3:
        bettis.append(0)
    betti_list.append(bettis)

for i in range(1, len(persistences)):
    dgm_prev = persistences[i - 1][1]
    dgm_curr = persistences[i][1]
    W = persim.wasserstein(dgm_prev, dgm_curr)
    wasserstein_list.append(W)

betti_df = pd.DataFrame(
    betti_list,
    columns=["Betti0", "Betti1", "Betti2"],
    index=df_finance.index[window_wasserstein - 1:]
)
wasserstein_df = pd.DataFrame(
    {"Wasserstein": wasserstein_list},
    index=df_finance.index[window_wasserstein:]
)

df_finance["Wasserstein"] = wasserstein_df.reindex(df_finance.index)["Wasserstein"]

for col in ["Betti0", "Betti1", "Betti2"]:
    df_finance[col] = betti_df.reindex(df_finance.index)[col]

IndexError: tuple index out of range

In [ ]:
df_finance = df_finance[sorted(df_finance.columns)]
df_finance

,AAPL_Close,AAPL_High,AAPL_Log_Return,AAPL_Low,AAPL_Open,AAPL_RSI_14,AAPL_Volatility_21,AAPL_Volume,AAPL_Volume_Z,ABBV_Close,...,ZBRA_Volume_Z,ZTS_Close,ZTS_High,ZTS_Log_Return,ZTS_Low,ZTS_Open,ZTS_RSI_14,ZTS_Volatility_21,ZTS_Volume,ZTS_Volume_Z
Date,,,,,,,,,,,,,,,,,,,,,
2025-09-16,238.149994,241.220001,0.006107,236.320007,237.179993,NaN,NaN,63421100,NaN,214.712112,...,NaN,147.460007,148.570007,0.008240,146.289993,146.289993,NaN,NaN,2664900,NaN
2025-09-17,238.990005,240.100006,0.003521,237.729996,238.970001,NaN,NaN,46508000,NaN,219.229553,...,NaN,146.929993,149.539993,-0.003601,146.559998,147.500000,NaN,NaN,1958300,NaN
2025-09-18,237.880005,241.199997,-0.004655,236.649994,239.970001,NaN,NaN,44249600,NaN,220.401108,...,NaN,147.100006,148.039993,0.001156,145.149994,146.300003,NaN,NaN,2739400,NaN
2025-09-19,245.500000,246.300003,0.031531,240.210007,241.229996,NaN,NaN,163741300,NaN,220.877670,...,NaN,145.880005,147.750000,-0.008328,145.300003,147.750000,NaN,NaN,4455800,NaN
2025-09-22,256.079987,256.640015,0.042193,248.119995,248.300003,NaN,NaN,105517400,NaN,220.996811,...,NaN,144.649994,146.710007,-0.008467,144.350006,146.550003,NaN,NaN,2044700,NaN
2025-09-23,254.429993,257.339996,-0.006464,253.580002,255.880005,NaN,NaN,60275200,NaN,221.016663,...,NaN,142.610001,146.179993,-0.014203,141.529999,143.250000,NaN,NaN,3679500,NaN
2025-09-24,252.309998,255.740005,-0.008367,251.039993,255.220001,NaN,NaN,42303700,NaN,218.415421,...,NaN,141.669998,144.009995,-0.006613,140.539993,141.619995,NaN,NaN,4224500,NaN
2025-09-25,256.869995,257.170013,0.017912,251.710007,253.210007,NaN,NaN,55202100,NaN,216.975800,...,NaN,141.130005,142.000000,-0.003819,139.339996,141.330002,NaN,NaN,3058500,NaN
2025-09-26,255.460007,257.600006,-0.005504,253.779999,254.100006,NaN,NaN,46076300,NaN,219.030991,...,NaN,143.500000,143.789993,0.016654,141.270004,141.860001,NaN,NaN,2575200,NaN


In [ ]:
df_finance.columns = pd.MultiIndex.from_tuples(
    [(col.split("_")[0], "_".join(col.split("_")[1:])) for col in df_finance.columns]
)

train_features = ['Close', 'High', 'Low', 'Open', 'RSI_14', 'Volatility_21', 'Volume', 'Volume_Z', 'Sentiment']

scaler = MinMaxScaler()

In [ ]:
df_finance

ATO                                               \
                  Open        High         Low       Close   Volume   
Date                                                                  
2025-09-16  164.610001  165.750000  162.899994  163.210007  1182300   
2025-09-17  163.570007  164.869995  162.529999  163.449997  1052400   
2025-09-18  161.130005  163.119995  160.100006  162.440002  1004300   
2025-09-19  162.779999  163.940002  161.449997  162.929993  1605700   
2025-09-22  162.720001  163.600006  162.199997  162.960007   778100   
2025-09-23  162.710007  166.490005  162.699997  166.339996   822200   
2025-09-24  166.339996  167.240005  166.080002  166.770004   621500   
2025-09-25  166.919998  168.860001  166.000000  166.460007   758700   
2025-09-26  167.679993  169.960007  167.039993  168.449997   687300   
2025-09-29  168.899994  169.699997  167.380005  169.550003   753100   
2025-09-30  169.720001  171.199997  169.369995  170.750000   866700   
2025-10-01  171.160004  171.160004  169.089996  169.380005   851400   
2025-10-02  168.720001  169.119995  167.619995  168.850006   578700   
2025-10-03  168.899994  170.990005  168.899994  170.190002   607200   
2025-10-06  170.639999  173.550003  170.190002  173.320007   843900   
2025-10-07  174.389999  175.490005  173.539993  174.910004   643500   
2025-10-08  175.570007  176.110001  173.220001  175.750000   740300   
2025-10-09  176.610001  177.009995  174.720001  174.929993   501000   
2025-10-10  175.910004  177.830002  175.289993  176.479996   639800   
2025-10-13  175.750000  176.919998  174.130005  175.539993   475400   
2025-10-14  176.210007  177.949997  175.410004  177.649994   610500   
2025-10-15  177.820007  179.699997  177.649994  179.190002   578300   
2025-10-16  178.630005  179.600006  176.149994  176.369995  1062100   
2025-10-17  176.869995  177.229996  175.020004  176.410004  1075000   
2025-10-20  177.570007  178.070007  175.309998  177.740005   654500   
2025-10-21  178.210007  178.490005  176.440002  177.039993   692300   
2025-10-22  177.699997  177.919998  176.139999  177.100006   729900   

                  FIS                                            ...  \
                 Open       High        Low      Close   Volume  ...   
Date                                                             ...   
2025-09-16  66.730003  66.949997  65.660004  66.440002  2989700  ...   
2025-09-17  66.540001  68.150002  66.320000  67.150002  4000800  ...   
2025-09-18  67.250000  67.320000  66.059998  66.239998  3720900  ...   
2025-09-19  66.599998  66.959999  65.029999  65.110001  6951500  ...   
2025-09-22  64.900002  65.080002  64.260002  64.650002  5799900  ...   
2025-09-23  64.580002  64.650002  63.200001  63.369999  4738100  ...   
2025-09-24  63.209999  64.300003  63.000000  64.129997  4875300  ...   
2025-09-25  64.120003  65.000000  63.650002  63.790001  4264900  ...   
2025-09-26  63.889999  64.339996  63.660000  64.239998  4640500  ...   
2025-09-29  64.480003  66.000000  64.110001  65.779999  4817800  ...   
2025-09-30  66.690002  67.129997  65.330002  65.940002  4396600  ...   
2025-10-01  65.870003  66.540001  65.370003  66.510002  3969700  ...   
2025-10-02  66.330002  67.250000  66.120003  66.449997  4130400  ...   
2025-10-03  66.419998  67.459999  66.180000  66.709999  2715900  ...   
2025-10-06  66.800003  67.300003  66.180000  67.150002  3526900  ...   
2025-10-07  67.800003  69.139999  67.459999  68.510002  3789000  ...   
2025-10-08  68.540001  68.680000  67.690002  68.129997  3515600  ...   
2025-10-09  68.529999  68.820000  67.940002  68.570000  2828100  ...   
2025-10-10  68.680000  69.040001  66.830002  66.870003  3158500  ...   
2025-10-13  67.029999  67.470001  66.230003  67.309998  2734500  ...   
2025-10-14  67.070000  68.500000  67.059998  67.739998  3666300  ...   
2025-10-15  67.660004  67.779999  66.339996  66.559998  2608700  ...   
2025-10-16  66.589996  66.989998  65.540001  65.809998  3215400  ...   
2025-10-17  65.9

In [ ]:
# Cell 6: Make predictions for each ticker
window_prediction = 30
predictions_dict = {}

for ticker in available_tickers[:10]:  # Process first 10 tickers as example
    try:
        print(f"Processing {ticker}...")
        
        # Get ticker data
        df_ticker_finance = df_finance[ticker]
        
        # Check if all required features are available
        available_features = [f for f in train_features if f in df_ticker_finance.columns]
        
        if len(available_features) < len(train_features):
            print(f"Skipping {ticker}: Missing features {set(train_features) - set(available_features)}")
            continue
            
        df_train = df_ticker_finance[available_features].copy()
        
        # Add Wasserstein distance
        df_train = df_train.merge(df_finance["Wasserstein"].loc[df_train.index], left_index=True, right_index=True)
        
        # Scale features
        df_train[available_features] = scaler.fit_transform(df_train[available_features])
        
        # Prepare sequences for prediction
        if len(df_train) >= window_prediction:
            # Get the most recent sequence
            last_sequence = df_train[available_features].iloc[-window_prediction:].values
            last_sequence = last_sequence.reshape(1, window_prediction, len(available_features))
            
            # Make prediction
            prediction = model.predict(last_sequence, verbose=0)[0][0]
            
            # Get the latest actual log return for comparison
            latest_log_return = df_ticker_finance["Log_Return"].iloc[-1]
            
            predictions_dict[ticker] = {
                'predicted_log_return': prediction,
                'latest_actual_log_return': latest_log_return,
                'prediction_date': df_train.index[-1]
            }
            
            print(f"{ticker}: Predicted = {prediction:.6f}, Actual = {latest_log_return:.6f}")
        else:
            print(f"Skipping {ticker}: Not enough data points")
            
    except Exception as e:
        print(f"Error processing {ticker}: {e}")
        continue

print(f"\nPredictions completed for {len(predictions_dict)} tickers")

In [ ]:
# Cell 7: Display results and save predictions
# Create results dataframe
results_df = pd.DataFrame.from_dict(predictions_dict, orient='index')
results_df = results_df.sort_values('predicted_log_return', ascending=False)

print("Top 10 predicted positive returns:")
print(results_df.head(10))

print("\nTop 10 predicted negative returns:")
print(results_df.tail(10))

# Save predictions
results_df.to_csv("latest_predictions.csv")
print(f"\nPredictions saved to latest_predictions.csv")

# Display summary statistics
print(f"\nSummary Statistics:")
print(f"Average predicted return: {results_df['predicted_log_return'].mean():.6f}")
print(f"Standard deviation: {results_df['predicted_log_return'].std():.6f}")
print(f"Min predicted return: {results_df['predicted_log_return'].min():.6f}")
print(f"Max predicted return: {results_df['predicted_log_return'].max():.6f}")